In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from scipy.sparse import hstack, csr_matrix
from sklearn.cluster import HDBSCAN

from sklearn.cluster import KMeans
from sklearn.metrics import make_scorer, silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import pos_tag
from spacy.lang.en.stop_words import STOP_WORDS as SPACY_STOP_WORDS

from mpl_toolkits.mplot3d import Axes3D # Required for 3D projection

import gensim
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')

nltk.data.path.append("./data")
nltk.download('punkt', download_dir="./data")
nltk.download("punkt_tab", download_dir="./data")
nltk.download("averaged_perceptron_tagger", download_dir="./data")
nltk.download("averaged_perceptron_tagger_eng", download_dir="./data")
nltk.download('stopwords', download_dir="./data")
nltk.download('wordnet', download_dir="./data")

import text_mining_utils as tmu

In [ ]:

df = pd.read_csv("./data/spunout_data.csv", encoding='utf-8-sig')

# Ensure titles are strings and handle potential NaN values by replacing them with empty strings
df['Title'] = df['Title'].fillna('').astype(str)

# Remove rows where content extraction failed (marked as 'N/A' by the scraper)
# Also drop rows where Content is actually NaN
df = df[df['Content'] != 'N/A']
df = df.dropna(subset=['Content'])

# Concatenate the Title with the Content
# We add a space in between to prevent the last word of the title merging with the first word of the body
df['Content'] = df['Title'] + " " + df['Content']

In [ ]:
# Text Preprocessing

# POS Tag Mapping Helper
# NLTK's POS tagger returns "Treebank" tags (e.g., NN, VB, JJ).
# The WordNetLemmatizer requires "WordNet" constants (e.g., 'n', 'v', 'a').
# We need a function to map between them.
def get_wordnet_pos(treebank_tag):
    """
    Maps NLTK POS tags to WordNet POS tags.
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        # Default to Noun if unknown (handles many abbreviations/slang)
        return wordnet.NOUN

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Load the Standard Library Stopwords (SpaCy is preferred over NLTK as it is more comprehensive)
# This automatically covers:
# - Pronouns ("he", "she", "they", "its", "whose", "whom")
# - Determiners ("this", "that", "these", "those")
# - Basic function words
stop_words = set(SPACY_STOP_WORDS)

# Extended Domain-Specific Stopwords for SpunOut.ie
# Includes site-specific noise, generic web terms, and common filler words.
domain_stopwords = [
    # Site & Web specific (Libraries don't know 'spunout' is noise)
    'spunout', 'spun', 'out', 'ie', 'ireland', 'irish', 'www', 'http', 'https', 'com', 
    'copyright', 'privacy', 'policy', 'terms', 'conditions', 'login', 'sign', 'register',
    
    # Scraping / HTML / Metadata Artifacts
    'page', 'section', 'footer', 'header', 'sidebar', 'widget', 'nav', 'advertisement', 'ad',
    'promo', 'cookie', 'script', 'javascript', 'css', 'html', 'body', 'main', 'published', 'updated',
    'author', 'post', 'article', 'url', 'permalink',
    
    # Generic Advice / High Frequency Verbs (Context specific noise)
    # Libraries consider these content words, but in an advice corpus they are fillers
    'day', 'new', 'good', 'bad',
    'check', 'try', 'keep',
    'like', 'just', 'get', 'also', 'would', 'could', 'one', 'make', 'use', 'way', 'well', 
    'time', 'know', 'need', 'really', 'thing', 'think', 'much', 'even', 'still', 'another', 
    'every', 'go', 'want', 'take', 'find', 'look', 'come', 'year', 'old', 'may', 'might',
    
    # Interaction / Navigation
    'click', 'read', 'link', 'menu', 'comment', 'reply',
    
    # Text Slang / Filler
    'u', 'ur', 'im', 'dont', 'cant', 'wont', 'oh', 'ok', 'please', 'thanks', 'thank', 'yes', 'no'
]

# Merge the standard library list with your custom list
stop_words.update(domain_stopwords)

# DATQ CLEANING FUNCTION
def clean_text(text):
    """
    Refined text cleaning function.
    Includes explicit removal of separator artifacts (e.g., ___, ///), 
    HTML/URL removal, Lemmatization, and Stopword filtering.
    """
    # 1. Ensure text is string and lowercase
    text = str(text).lower()
    
    # 2. Remove URLs and HTML tags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'<.*?>', ' ', text)
    
    # 3. NEW: Explicitly remove common page break/artifact separators
    # Matches 2 or more underscores, dashes, or dots (e.g., "___", "---", "...")
    text = re.sub(r'[\_\-\.]{1,}', ' ', text)
    # Remove standalone slashes (forward or backward)
    text = re.sub(r'[/\\]', ' ', text)

    # 4. Remove apostrophes to unify contractions (e.g., "don't" -> "dont")
    text = re.sub(r"\'", "", text)
    
    # 5. Remove all non-letter characters (except spaces) - Final Polish
    # This removes remaining symbols like @, #, $, %, &, *, etc.
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 6. Remove single characters that are surrounded by spaces
    # This cleans up leftover fragments like " a " or " b " that usually hold no meaning
    text = re.sub(r'\s+[a-z]\s+', ' ', text)
    
    # 7. Tokenize (split into words)
    words = text.split()
    
    # 8. POS Tagging
    # Creates a list of tuples: [('checking', 'VBG'), ('email', 'NN')]
    # Note: This step is the slowest part of the pipeline.
    tagged_words = pos_tag(words)
    
    # 9. Context-Aware Lemmatization
    lemmatized_words = []
    for word, tag in tagged_words:
        # Map the Treebank tag to WordNet tag
        wn_tag = get_wordnet_pos(tag)
        
        # Lemmatize using the specific Part of Speech
        # If the word is "running" and tag is Verb, it becomes "run"
        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        lemmatized_words.append(lemma)
    
    # 10. Filter Stopwords and Short Words
    # We filter AFTER lemmatization to ensure we catch the root forms
    filtered_words = [word for word in lemmatized_words if word not in stop_words and len(word) > 2]
    
    return " ".join(filtered_words)

In [ ]:
# PIPELINE EXECUTION

# Apply Text Cleaning 
print("Applying Text Cleaning (POS + Lemmatization)...")
df['Clean_Content'] = df['Content'].apply(clean_text)

# Define Features (X) and Target (y)
# X_raw represents the raw combined text data
# X represents the preprocessed text data
X_raw = df['Content']
X = df['Clean_Content']
# y represents the labels (categories) for validation/comparison
y = df['Category']
t = df['Topic']

In [ ]:
# TF-IDF VECTORIZATION
print("Vectorizing with TF-IDF...")

# Initialize TF-IDF Vectorizer with advanced parameters
# We use the 'stop_words' variable created in your preprocessing step
# (which merged standard English words with your custom domain list)
tfidf_vectorizer = TfidfVectorizer(
    # Increase max_features to 20,000 to accommodate the explosion of trigrams
    max_features=20000,           
    
    # Expand to Trigrams: Essential for separating specific phrases 
    # (e.g., distinguishing "mental health" from "mental health act")
    ngram_range=(1, 3),
    
    # Frequency Filtering: 
    # min_df=3 removes noise/typos (words appearing in fewer than 3 docs)
    min_df=3,                     
    
    # max_df=0.7 removes "glue" words appearing in more than 70% of docs
    max_df=0.7,                    
    
    # Logarithmic scaling: Dampens the effect of words appearing 100 times vs 10 times
    sublinear_tf=True,            
    
    # Use L2 Norm: Ensures all document vectors have the same length
    norm='l2'                     
)

# Fit and transform the cleaned content
X_tfidf_matrix = tfidf_vectorizer.fit_transform(X)

print(f"TF-IDF Shape: {X_tfidf_matrix.shape}")

In [ ]:
def find_optimal_k(X, variance_threshold=0.90, max_components=5000):
    """
    Finds the smallest number of components (K) that explains 
    a specific percentage of the variance in the data.
    """
    n_samples, n_features = X.shape
    
    # Set a safe upper limit for the search (to prevent memory crashes)
    # Usually we don't need more components than we have samples
    search_limit = min(n_samples, max_components, n_features)
    
    print(f"Analyzing variance for up to {search_limit} components...")
    
    # 1. Fit SVD with the search limit
    svd_analyzer = TruncatedSVD(n_components=search_limit, random_state=42)
    svd_analyzer.fit(X)
    
    # 2. Calculate Cumulative Explained Variance
    # This array tells us how much "information" is preserved as we add components
    explained_variance_ratio = svd_analyzer.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    # 3. Find the 'k' where we hit the variance threshold
    # We add 1 because indices are 0-based
    k_indices = np.where(cumulative_variance >= variance_threshold)[0]
    
    if len(k_indices) > 0:
        optimal_k = k_indices[0] + 1
        variance_acquired = cumulative_variance[optimal_k - 1]
    else:
        # If we didn't reach the threshold within the limit, use the max limit
        optimal_k = search_limit
        variance_acquired = cumulative_variance[-1]
        print(f"Warning: Threshold {variance_threshold*100}% not reached within {search_limit} components.")


    print(f"Target Variance:   {variance_threshold*100}%")
    print(f"Optimal K found:   {optimal_k}")
    print(f"Actual Variance:   {variance_acquired:.4f} ({variance_acquired*100:.2f}%)")

    
    return optimal_k



# Find the best K automatically (aiming for 90% variance)
# You can adjust variance_threshold to 0.95 for stricter precision, or 0.85 for speed
best_k = find_optimal_k(X_tfidf_matrix, variance_threshold=0.90)

# Apply TruncatedSVD with the calculated best_k
print(f"\nApplying TruncatedSVD with K={best_k}...")
svd_final = TruncatedSVD(n_components=best_k, random_state=42)

X_tfidf_reduced_matrix = svd_final.fit_transform(X_tfidf_matrix)

print(f"Final Reduced Matrix Shape: {X_tfidf_reduced_matrix.shape}")

In [ ]:
from sklearn.preprocessing import Normalizer

# SHAPE THE TRAINING DATA: Normalize to Unit Length
# This converts Euclidean distance into Cosine Similarity logic.
# It prevents "Big" clusters from swallowing "Small" ones due to magnitude.
normalizer = Normalizer()
X_tfidf_matrix_normalized = normalizer.fit_transform(X_tfidf_reduced_matrix)

print("\nData normalized for Spherical K-Means...")

In [ ]:
import os
from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

# Lists to store evaluation metrics
db_scores = []
sil_scores = []

# Use normalized data from your pipeline
X_data = X_tfidf_matrix_normalized.copy()

print("Evaluating Agglomerative Clustering from K=2 to K=10...\n")

for k in range(2, 10):
    # Initialize Agglomerative clustering
    agg = AgglomerativeClustering(
        n_clusters=k,
        metric='euclidean',
        linkage='ward'    # good default — requires euclidean
    )
    
    # Fit & predict labels
    labels = agg.fit_predict(X_data)
    
    # Compute metrics
    db_index = davies_bouldin_score(X_data, labels)
    sil_score = silhouette_score(X_data, labels)
    
    db_scores.append(db_index)
    sil_scores.append(sil_score)

    print(f"K={k}: Davies-Bouldin={db_index:.4f} | Silhouette={sil_score:.4f}")

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(range(2, 10), db_scores, marker='o', label='Davies-Bouldin', color='red')
ax.plot(range(2, 10), sil_scores, marker='x', label='Silhouette', color='blue')

ax.set_xlabel('Number of clusters (k)')
ax.set_ylabel('Score')
# ax.set_title('Agglomerative Clustering Validation Metrics')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.7)

# Change this to whatever folder name you want
folder_name = "./practical_assessment_adsah_6014_2_web_content_mining/images/"
file_name = "agglomerative_clustering_clusters_dbouldin_silhouette_euclidean.png"

# Create the folder if it doesn't exist yet
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Combine them into a full path (handles Windows/Mac differences automatically)
full_path = os.path.join(folder_name, file_name)

plt.savefig(full_path, dpi=300, bbox_inches='tight')

print(f"\nPlot successfully saved to: {full_path}")

plt.show()

In [ ]:
import os
from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt

# Lists to store evaluation metrics
db_scores = []
sil_scores = []

# Use normalized data from your pipeline
X_data = X_tfidf_matrix_normalized.copy()

print("Evaluating Agglomerative Clustering from K=2 to K=10...\n")

for k in range(2, 10):
    # Initialize Agglomerative clustering
    agg = AgglomerativeClustering(
        n_clusters=k,
        metric='cosine',
        linkage='complete'    # good for cosine — requires pre-normalized data
    )
    
    # Fit & predict labels
    labels = agg.fit_predict(X_data)
    
    # Compute metrics
    db_index = davies_bouldin_score(X_data, labels)
    sil_score = silhouette_score(X_data, labels)
    
    db_scores.append(db_index)
    sil_scores.append(sil_score)

    print(f"K={k}: Davies-Bouldin={db_index:.4f} | Silhouette={sil_score:.4f}")

# Plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(range(2, 10), db_scores, marker='o', label='Davies-Bouldin', color='red')
ax.plot(range(2, 10), sil_scores, marker='x', label='Silhouette', color='blue')

ax.set_xlabel('Number of clusters (k)')
ax.set_ylabel('Score')
# ax.set_title('Agglomerative Clustering Validation Metrics')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.7)

# Change this to whatever folder name you want
folder_name = "./practical_assessment_adsah_6014_2_web_content_mining/images/"
file_name = "agglomerative_clustering_clusters_dbouldin_silhouette_cosine.png"

# Create the folder if it doesn't exist yet
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Combine them into a full path (handles Windows/Mac differences automatically)
full_path = os.path.join(folder_name, file_name)

plt.savefig(full_path, dpi=300, bbox_inches='tight')

print(f"\nPlot successfully saved to: {full_path}")

plt.show()


In [ ]:
# Instantiate Agglomerative Clustering
agg = AgglomerativeClustering(
    n_clusters=7,
    metric='euclidean',
    linkage='ward'    # good default — requires euclidean
)

param_grid = {
    # 1. compute_distances: 
    # Calculates the distances between merged clusters. 
    # False = Faster (Default)
    # True  = Required if you want to plot a Dendrogram later
    'compute_distances': [False, True],
    
    # 2. compute_full_tree:
    # Determines if the algorithm stops merging as soon as it hits 7 clusters ('auto') 
    # or builds the entire hierarchy first (True).
    # 'auto' = Faster, lower memory
    # True   = Slower, higher memory, necessary for very specific distance calculations
    'compute_full_tree': ['auto', True]
}

# Grid Search
# Note: AgglomerativeClustering is memory-intensive. 
# If this crashes with MemoryError, reduce 'cv' to 3.
print("Running Grid Search for Agglomerative Clustering...")
print("Note: This method merges all data (no noise) but can be slower on large datasets.")

agg_grid_search = GridSearchCV(
    agg, 
    param_grid=param_grid, 
    scoring=make_scorer(silhouette_score),
    cv=3,         # 3-fold Cross Validation
    n_jobs=-1,    # Use all cores (GridSearchCV level)
    verbose=1
)

# Fit on NORMALIZED data
agg_grid_search.fit(X_tfidf_matrix_normalized)

print("\nBest Parameters Found:")
print(agg_grid_search.best_params_)

In [ ]:
# Fitting the Final Model
best_agg = agg_grid_search.best_estimator_

# Convert sparse TF-IDF to a dense DataFrame (optional: keep sparse for memory efficiency)
agg_results_df = pd.DataFrame(X_tfidf_matrix_normalized)

# Add cluster labels
agg_results_df['cluster_label'] = best_agg.labels_

# Add true original website labels (Category)
agg_results_df['true_label'] = y

# Print how clusters correspond to categories
print("\nCluster Composition:")
print(agg_results_df.groupby('cluster_label')['true_label'].value_counts().to_string())

In [ ]:
# Fitting the Final Model
best_agg = agg_grid_search.best_estimator_

# Convert sparse TF-IDF to a dense DataFrame (optional: keep sparse for memory efficiency)
agg_results_df = pd.DataFrame(X_tfidf_matrix_normalized)

# Add cluster labels
agg_results_df['cluster_label'] = best_agg.labels_

# Add true original website labels (Category)
agg_results_df['true_label'] = y

agg_results_df['topics'] = t

# Group by the three dimensions and count the occurrences
composition = agg_results_df.groupby(['cluster_label', 'true_label', 'topics']).size()

print("\nDetailed Cluster Composition (Cluster > Category > Topic):")
print(composition.to_string())

In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.decomposition import TruncatedSVD

print("\nGenerating 3D Cluster Visualization with Plotly...")

# Reduce dimensions to 3D
svd = TruncatedSVD(n_components=3, random_state=43)
X_svd = svd.fit_transform(X_tfidf_matrix_normalized)

# Create DataFrame
plot_df = pd.DataFrame(X_svd, columns=['Component 1', 'Component 2', 'Component 3'])

# FIX: Keep labels as INTEGERS (numbers), not strings
# This allows us to use the Viridis gradient
plot_df['Cluster'] = best_agg.labels_

# Plot using scatter_3d
fig = px.scatter_3d(
    plot_df,
    x='Component 1',
    y='Component 2',
    z='Component 3',
    color='Cluster',
    title=f'Interactive 3D Agglomerative Clusters<br><sub>TruncatedSVD Projection of TF-IDF Matrix</sub>',
    opacity=0.7,
    width=1000,
    height=800,
    # FIX: Use color_continuous_scale for gradients like Viridis
    color_continuous_scale='viridis'
)

# Update the layout for cleaner axis labels
fig.update_traces(marker=dict(size=3, line=dict(width=0)))
fig.update_layout(
    scene=dict(
        xaxis_title='Component 1',
        yaxis_title='Component 2',
        zaxis_title='Component 3'
    ),
    margin=dict(l=0, r=0, b=0, t=50) # Tight layout
)

fig.show()

In [ ]:
import math

# Feature names
feature_names = tfidf_vectorizer.get_feature_names_out()

# Number of clusters
n_clusters = best_agg.n_clusters  # this = 7

#  Dynamically determine grid size
cols = 2
rows = math.ceil(n_clusters / cols)

fig, axes = plt.subplots(rows, cols, figsize=(16, 5 * rows))
fig.suptitle('Top 20 Words per Cluster (by TF-IDF Score)', fontsize=20)

axes = axes.flatten()

for i in range(n_clusters):
    # 1. Docs in this cluster
    cluster_indices = np.where(best_agg.labels_ == i)[0]
    
    # 2. TF-IDF rows for cluster
    cluster_tfidf = X_tfidf_matrix[cluster_indices]
    
    # 3. Sum TF-IDF scores
    word_scores = np.array(cluster_tfidf.sum(axis=0)).flatten()
    
    # 4. Top 20 words
    top_indices = word_scores.argsort()[-20:][::-1]
    top_words = feature_names[top_indices]
    top_scores = word_scores[top_indices]
    
    # 5. DataFrame
    df_plot = pd.DataFrame({"Word": top_words, "Score": top_scores})

    # 6. Plot
    sns.barplot(ax=axes[i], data=df_plot, x='Score', y='Word', palette='viridis')
    axes[i].set_title(f'Cluster {i}', fontsize=14)
    axes[i].set_xlabel('Total TF-IDF Score')
    axes[i].set_ylabel('')

# Hide extra empty subplots
for j in range(n_clusters, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Change this to whatever folder name you want
folder_name = "./practical_assessment_adsah_6014_2_web_content_mining/images/"
file_name = "agglomerative_top_20_words_per_cluster.png"

# Create the folder if it doesn't exist yet
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Combine them into a full path (handles Windows/Mac differences automatically)
full_path = os.path.join(folder_name, file_name)

plt.savefig(full_path, dpi=300, bbox_inches='tight')

print(f"\nPlot successfully saved to: {full_path}")

plt.show()

In [ ]:
print("Top Words per Cluster (by TF-IDF Score):")
print("-" * 40)

for i in range(n_clusters):
    # (Steps A, B, C, D from your original code remain the same)
    cluster_indices = np.where(best_agg.labels_ == i)[0]
    cluster_tfidf = X_tfidf_matrix[cluster_indices]
    word_scores = np.array(cluster_tfidf.sum(axis=0)).flatten()
    top_indices = word_scores.argsort()[-20:][::-1]
    
    top_words = feature_names[top_indices]
    top_scores = word_scores[top_indices]
    
    # Text display logic
    print(f"\nCluster {i}:")
    # Join words with commas for a compact view
    print(", ".join(top_words))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

# Calculate centroids for all clusters
centroids = []
for i in range(n_clusters):
    mask = (best_agg.labels_ == i)
    centroids.append(X_tfidf_matrix_normalized[mask].mean(axis=0))

centroids = np.array(centroids)

# Compute similarity matrix between centroids
centroid_similarity = cosine_similarity(centroids)

# Plot as a Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    centroid_similarity, 
    annot=True, 
    cmap='coolwarm', 
    xticklabels=[f'C{i}' for i in range(n_clusters)],
    yticklabels=[f'C{i}' for i in range(n_clusters)],
    vmin=0, vmax=1
)
#plt.title('Inter-Cluster Similarity (Topic Overlap)')

# Change this to whatever folder name you want
folder_name = "./practical_assessment_adsah_6014_2_web_content_mining/images/"
file_name = "agglomerative_inter_cluster_similarity.png"

# Create the folder if it doesn't exist yet
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Combine them into a full path (handles Windows/Mac differences automatically)
full_path = os.path.join(folder_name, file_name)

plt.savefig(full_path, dpi=300, bbox_inches='tight')

print(f"\nPlot successfully saved to: {full_path}")

plt.show()

In [ ]:
# Create labels for the clusters (e.g., 'Cluster 0', 'Cluster 1', etc.)
cluster_labels = [f'Cluster {i}' for i in range(n_clusters)]

# Convert the numpy array into a Pandas DataFrame
# We use the labels for both the Index (rows) and Columns
df_similarity = pd.DataFrame(
    centroid_similarity, 
    index=cluster_labels, 
    columns=cluster_labels
)

# Round the values to 3 decimal places for readability
df_similarity = df_similarity.round(3)

# Print the text-based table
print("\nInter-Cluster Similarity Matrix:")
print(df_similarity)

In [ ]:
print("Identifying potential Outliers (weakest links)...\n")

for i in range(n_clusters):
    cluster_mask = (best_agg.labels_ == i)
    cluster_docs = X_tfidf_matrix_normalized[cluster_mask]
    
    # FIX: Reshape the centroid to be 2D (1 row, n_features columns)
    # .mean(axis=0) creates a 1D array. .reshape(1, -1) converts it to 2D.
    centroid = cluster_docs.mean(axis=0).reshape(1, -1)
    
    # Calculate similarities
    similarities = cosine_similarity(cluster_docs, centroid).flatten()
    
    # Find the document with the LOWEST similarity to the center
    worst_doc_local_index = similarities.argsort()[0]
    
    original_df_indices = np.where(best_agg.labels_ == i)[0]
    worst_original_index = original_df_indices[worst_doc_local_index]
    
    score = similarities[worst_doc_local_index]
    
    # Only print if the document is really far away (e.g., similarity < 0.1)
    if score < 0.1: 
        title = df['Title'].iloc[worst_original_index]
        print(f"CLUSTER {i} Outlier (Score: {score:.4f}):")
        print(f"  > {title}\n")